In [1]:
# LocalStack & boto3
# Working with DynamoDB in Python using LocalStack & Boto3

# All OK !

# Create a DynamoDB Table
# Performing CRUD Operations on DynamoDB Table
# Read Items
# Update Items
# Re Reading
# Delete Items
# Delete a DynamoDB Table


# boto3_DynamoDB_python

# Create a DynamoDB Table

import json
import logging
from datetime import date, datetime
import os

import boto3
from botocore.exceptions import ClientError

AWS_REGION = 'us-east-1'
AWS_PROFILE = 'localstack'
ENDPOINT_URL = os.environ.get('LOCALSTACK_ENDPOINT_URL')

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

boto3.setup_default_session(profile_name=AWS_PROFILE)

dynamodb_client = boto3.client(
    "dynamodb", region_name=AWS_REGION, endpoint_url=ENDPOINT_URL)


def json_datetime_serializer(obj):
    """
    Helper method to serialize datetime fields
    """
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError("Type %s not serializable" % type(obj))


def create_dynamodb_table(table_name):
    """
    Creates a DynamoDB table.
    """
    try:
        response = dynamodb_client.create_table(
            TableName=table_name,
            KeySchema=[
                {
                    'AttributeName': 'Name',
                    'KeyType': 'HASH'
                },
                {
                    'AttributeName': 'Email',
                    'KeyType': 'RANGE'
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'Name',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'Email',
                    'AttributeType': 'S'
                }
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 1,
                'WriteCapacityUnits': 1
            },
            Tags=[
                {
                    'Key': 'Name',
                    'Value': 'hands-on-cloud-dynamodb-table'
                }
            ])

    except ClientError:
        logger.exception('Could not create the table.')
        raise
    else:
        return response


def main():
    """
    Main invocation function.
    """
    table_name = 'hands-on-cloud-dynamodb-table'
    logger.info('Creating a DynamoDB table...')
    dynamodb = create_dynamodb_table(table_name)
    logger.info(
        f'DynamoDB table created: {json.dumps(dynamodb, indent=4, default=json_datetime_serializer)}')

if __name__ == '__main__':
    main()


2022-06-19 13:29:58,428: INFO: Found credentials in shared credentials file: ~/.aws/credentials
2022-06-19 13:29:58,921: INFO: Creating a DynamoDB table...
2022-06-19 13:30:44,495: INFO: DynamoDB table created: {
    "TableDescription": {
        "AttributeDefinitions": [
            {
                "AttributeName": "Name",
                "AttributeType": "S"
            },
            {
                "AttributeName": "Email",
                "AttributeType": "S"
            }
        ],
        "TableName": "hands-on-cloud-dynamodb-table",
        "KeySchema": [
            {
                "AttributeName": "Name",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "Email",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": "2022-06-19T13:30:44.167000-05:00",
        "ProvisionedThroughput": {
            "ReadCapacityUnits": 1,
            "WriteCapacityUnits": 1
 

In [3]:

# Performing CRUD Operations on DynamoDB Table

# Create, Read, Update and Delete (CRUD) operations on Items in DynamoDB Tables.

'''
Create Items
To add a new item to a table, you need to use the put_item() method from the Boto3 library.

The below example adds an item with Name and Email keys.
'''

# Create Items
import json
import logging
import os

import boto3
from botocore.exceptions import ClientError

AWS_REGION = 'us-east-1'
AWS_PROFILE = 'localstack'
ENDPOINT_URL = os.environ.get('LOCALSTACK_ENDPOINT_URL')

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

boto3.setup_default_session(profile_name=AWS_PROFILE)

dynamodb_resource = boto3.resource(
    "dynamodb", region_name=AWS_REGION, endpoint_url=ENDPOINT_URL)


def add_dynamodb_table_item(table_name, name, email):
    """
    adds a DynamoDB table.
    """
    try:
        table = dynamodb_resource.Table(table_name)
        response = table.put_item(
            Item={
                'Name': name,
                'Email': email
            }
        )

    except ClientError:
        logger.exception('Could not add the item to table.')
        raise
    else:
        return response


def main():
    """
    Main invocation function.
    """
    table_name = 'hands-on-cloud-dynamodb-table'
    name = 'hands-on-cloud'
    email = 'example@cloud.com'
    logger.info('Adding item...')
    dynamodb = add_dynamodb_table_item(table_name, name, email)
    logger.info(
        f'DynamoDB table item created: {json.dumps(dynamodb, indent=4)}')


if __name__ == '__main__':
    main()

2022-06-19 13:41:51,663: INFO: Found credentials in shared credentials file: ~/.aws/credentials
2022-06-19 13:41:52,046: INFO: Adding item...
2022-06-19 13:41:54,656: INFO: DynamoDB table item created: {
    "ConsumedCapacity": {
        "TableName": "hands-on-cloud-dynamodb-table",
        "CapacityUnits": 1.0
    },
    "ResponseMetadata": {
        "RequestId": "99PRMPBI1DA1EFBCSRA63IE06CEYDZ5JRZWB1PMI3UL4X902YKWI",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "content-length": "90",
            "x-amzn-requestid": "99PRMPBI1DA1EFBCSRA63IE06CEYDZ5JRZWB1PMI3UL4X902YKWI",
            "x-amz-crc32": "125036636",
            "access-control-allow-origin": "*",
            "access-control-allow-methods": "HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH",
            "access-control-allow-headers": "authorization,cache-control,content-length,content-md5,content-type,etag,location,x-amz-acl,x-amz-content-sha256,x-amz-d

In [6]:
# Read Items
# use the get_item() method to read an item from the hands-on-cloud-dynamodb-table table.
# specify the primary key value to read any row in hands-on-cloud-dynamodb-table.

# Read Items
import json
import logging
import os

import boto3
from botocore.exceptions import ClientError

AWS_REGION = 'us-east-1'
AWS_PROFILE = 'localstack'
ENDPOINT_URL = os.environ.get('LOCALSTACK_ENDPOINT_URL')

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

boto3.setup_default_session(profile_name=AWS_PROFILE)

dynamodb_resource = boto3.resource(
    "dynamodb", region_name=AWS_REGION, endpoint_url=ENDPOINT_URL)


def read_dynamodb_table_item(table_name, name, email):
    """
    Reads from a DynamoDB table.
    """
    try:
        table = dynamodb_resource.Table(table_name)
        response = table.get_item(
            Key={
                'Name': name,
                'Email': email
            }
        )

    except ClientError:
        logger.exception('Could not read the item from table.')
        raise
    else:
        return response


def main():
    """
    Main invocation function.
    """
    table_name = 'hands-on-cloud-dynamodb-table'
    name = 'hands-on-cloud'
    email = 'example@cloud.com'
    logger.info('Reading item...')
    dynamodb = read_dynamodb_table_item(table_name, name, email)
    logger.info(
        f'Item details: {json.dumps(dynamodb, indent=4)}')


if __name__ == '__main__':
    main()


2022-06-19 13:58:38,266: INFO: Found credentials in shared credentials file: ~/.aws/credentials
2022-06-19 13:58:38,491: INFO: Reading item...
2022-06-19 13:58:40,577: INFO: Item details: {
    "Item": {
        "phone_number": "123-456-1234",
        "Email": "example@cloud.com",
        "Name": "hands-on-cloud"
    },
    "ResponseMetadata": {
        "RequestId": "AP7B4SJT7A738MUOZT63LQM3JFGTWMXCMNKG3CKBNH4JM75HJI2J",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "content-length": "119",
            "x-amzn-requestid": "AP7B4SJT7A738MUOZT63LQM3JFGTWMXCMNKG3CKBNH4JM75HJI2J",
            "x-amz-crc32": "4152759509",
            "access-control-allow-origin": "*",
            "access-control-allow-methods": "HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH",
            "access-control-allow-headers": "authorization,cache-control,content-length,content-md5,content-type,etag,location,x-amz-acl,x-amz-content-sha256,x-a

In [5]:
# Update Items

# It needs to make changes to an existing available item, you’ll need to use the update_item() method.

# Update Items
import json
import logging
import os

import boto3
from botocore.exceptions import ClientError

AWS_REGION = 'us-east-1'
AWS_PROFILE = 'localstack'
ENDPOINT_URL = os.environ.get('LOCALSTACK_ENDPOINT_URL')

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

boto3.setup_default_session(profile_name=AWS_PROFILE)

dynamodb_resource = boto3.resource(
    "dynamodb", region_name=AWS_REGION, endpoint_url=ENDPOINT_URL)


def update_dynamodb_table_item(table_name, name, email, phone_number):
    """
    update the DynamoDB table item.
    """
    try:
        table = dynamodb_resource.Table(table_name)
        response = table.update_item(
            Key={
                'Name': name,
                'Email': email
            },
            UpdateExpression="set phone_number=:ph",
            ExpressionAttributeValues={
                ':ph': phone_number
            }
        )

    except ClientError:
        logger.exception('Could not update the item.')
        raise
    else:
        return response


def main():
    """
    Main invocation function.
    """
    table_name = 'hands-on-cloud-dynamodb-table'
    name = 'hands-on-cloud'
    email = 'example@cloud.com'
    phone_number = '123-456-1234'
    logger.info('updateing item...')
    dynamodb = update_dynamodb_table_item(
        table_name, name, email, phone_number)
    logger.info(
        f'Item details: {json.dumps(dynamodb, indent=4)}')


if __name__ == '__main__':
    main()


2022-06-19 13:57:21,738: INFO: Found credentials in shared credentials file: ~/.aws/credentials
2022-06-19 13:57:21,890: INFO: updateing item...
2022-06-19 13:57:25,348: INFO: Item details: {
    "ConsumedCapacity": {
        "TableName": "hands-on-cloud-dynamodb-table",
        "CapacityUnits": 1.0
    },
    "ResponseMetadata": {
        "RequestId": "D6GXRXKIITUHGMPT0I50HE24FV79AB6ZV286CCV78PHBGMOFY20Y",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "content-length": "90",
            "x-amzn-requestid": "D6GXRXKIITUHGMPT0I50HE24FV79AB6ZV286CCV78PHBGMOFY20Y",
            "x-amz-crc32": "125036636",
            "access-control-allow-origin": "*",
            "access-control-allow-methods": "HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH",
            "access-control-allow-headers": "authorization,cache-control,content-length,content-md5,content-type,etag,location,x-amz-acl,x-amz-content-sha256,x-amz-date,x-amz-re

In [7]:
# Re Reading

def main():
    """
    Main invocation function.
    """
    table_name = 'hands-on-cloud-dynamodb-table'
    name = 'hands-on-cloud'
    email = 'example@cloud.com'
    logger.info('Reading item...')
    dynamodb = read_dynamodb_table_item(table_name, name, email)
    logger.info(
        f'Item details: {json.dumps(dynamodb, indent=4)}')


if __name__ == '__main__':
    main()

2022-06-19 13:59:19,091: INFO: Reading item...
2022-06-19 13:59:21,211: INFO: Item details: {
    "Item": {
        "phone_number": "123-456-1234",
        "Email": "example@cloud.com",
        "Name": "hands-on-cloud"
    },
    "ResponseMetadata": {
        "RequestId": "B6QRO0RGBBTW4YTHUHWT6BLFQGZ63XL2UHFSNVB9SGQV0ACOA24P",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "content-length": "119",
            "x-amzn-requestid": "B6QRO0RGBBTW4YTHUHWT6BLFQGZ63XL2UHFSNVB9SGQV0ACOA24P",
            "x-amz-crc32": "4152759509",
            "access-control-allow-origin": "*",
            "access-control-allow-methods": "HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH",
            "access-control-allow-headers": "authorization,cache-control,content-length,content-md5,content-type,etag,location,x-amz-acl,x-amz-content-sha256,x-amz-date,x-amz-request-id,x-amz-security-token,x-amz-tagging,x-amz-target,x-amz-user-agent,x-amz-

In [8]:

# Delete Items

# To delete an item from the table, It needs to use the delete_item() method to delete one item by providing its primary key

# Delete Items
import json
import logging
import os

import boto3
from botocore.exceptions import ClientError

AWS_REGION = 'us-east-1'
AWS_PROFILE = 'localstack'
ENDPOINT_URL = os.environ.get('LOCALSTACK_ENDPOINT_URL')

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

boto3.setup_default_session(profile_name=AWS_PROFILE)

dynamodb_resource = boto3.resource(
    "dynamodb", region_name=AWS_REGION, endpoint_url=ENDPOINT_URL)


def delete_dynamodb_table_item(table_name, name, email):
    """
    Deletes the DynamoDB table item.
    """
    try:
        table = dynamodb_resource.Table(table_name)
        response = table.delete_item(
            Key={
                'Name': name,
                'Email': email
            }
        )

    except ClientError:
        logger.exception('Could not delete the item.')
        raise
    else:
        return response


def main():
    """
    Main invocation function.
    """
    table_name = 'hands-on-cloud-dynamodb-table'
    name = 'hands-on-cloud'
    email = 'example@cloud.com'
    logger.info('Deleteing item...')
    dynamodb = delete_dynamodb_table_item(
        table_name, name, email)
    logger.info(
        f'Details: {json.dumps(dynamodb, indent=4)}')


if __name__ == '__main__':
    main()


2022-06-19 14:00:08,542: INFO: Found credentials in shared credentials file: ~/.aws/credentials
2022-06-19 14:00:08,674: INFO: Deleteing item...
2022-06-19 14:00:10,864: INFO: Details: {
    "ConsumedCapacity": {
        "TableName": "hands-on-cloud-dynamodb-table",
        "CapacityUnits": 1.0
    },
    "ResponseMetadata": {
        "RequestId": "NH22PFBQ72TUXVS9TM66JHQQVD83L2TINQMORK1E8WB2N14CMEZ3",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "content-length": "90",
            "x-amzn-requestid": "NH22PFBQ72TUXVS9TM66JHQQVD83L2TINQMORK1E8WB2N14CMEZ3",
            "x-amz-crc32": "125036636",
            "access-control-allow-origin": "*",
            "access-control-allow-methods": "HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH",
            "access-control-allow-headers": "authorization,cache-control,content-length,content-md5,content-type,etag,location,x-amz-acl,x-amz-content-sha256,x-amz-date,x-amz-request

In [9]:
# Delete a DynamoDB Table

# To delete a DynamoDB table, you need to use the delete_table() method.

# Delete a DynamoDB Table
import json
import logging
from datetime import date, datetime
import os

import boto3
from botocore.exceptions import ClientError

AWS_REGION = 'us-east-1'
AWS_PROFILE = 'localstack'
ENDPOINT_URL = os.environ.get('LOCALSTACK_ENDPOINT_URL')

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

boto3.setup_default_session(profile_name=AWS_PROFILE)

dynamodb_client = boto3.client(
    "dynamodb", region_name=AWS_REGION, endpoint_url=ENDPOINT_URL)


def json_datetime_serializer(obj):
    """
    Helper method to serialize datetime fields
    """
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError("Type %s not serializable" % type(obj))


def delete_dynamodb_table(table_name):
    """
    Deletes the DynamoDB table.
    """
    try:
        response = dynamodb_client.delete_table(
            TableName=table_name
        )

    except ClientError:
        logger.exception('Could not delete the table.')
        raise
    else:
        return response


def main():
    """
    Main invocation function.
    """
    table_name = 'hands-on-cloud-dynamodb-table'
    logger.info('Deleteing DynamoDB table...')
    dynamodb = delete_dynamodb_table(table_name)
    logger.info(
        f'Details: {json.dumps(dynamodb, indent=4, default=json_datetime_serializer)}')


if __name__ == '__main__':
    main()



2022-06-19 14:01:20,343: INFO: Found credentials in shared credentials file: ~/.aws/credentials
2022-06-19 14:01:20,500: INFO: Deleteing DynamoDB table...
2022-06-19 14:01:23,446: INFO: Details: {
    "TableDescription": {
        "AttributeDefinitions": [
            {
                "AttributeName": "Name",
                "AttributeType": "S"
            },
            {
                "AttributeName": "Email",
                "AttributeType": "S"
            }
        ],
        "TableName": "hands-on-cloud-dynamodb-table",
        "KeySchema": [
            {
                "AttributeName": "Name",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "Email",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": "2022-06-19T13:30:44.167000-05:00",
        "ProvisionedThroughput": {
            "LastIncreaseDateTime": "1969-12-31T19:00:00-05:00",
            "LastDecreas